In [1]:
import numpy as np

In [2]:
def func(A, b):
    A = A.astype(float).copy()
    b = b.astype(float).copy()
    n = A.shape[0]

    col_order = np.arange(n)
    det = 1.0

    for k in range(n):
        sub_matrix = np.abs(A[k:, k:])
        flat_index = np.argmax(sub_matrix)
        i_max = flat_index // (n - k) + k
        j_max = flat_index % (n - k) + k

        if abs(A[i_max, j_max]) < 1e-10:
            raise ValueError("Matrix is singular; no unique solution exists.")

        if i_max != k:
            A[[k, i_max]] = A[[i_max, k]]
            b[[k, i_max]] = b[[i_max, k]]
            det *= -1

        if j_max != k:
            A[:, [k, j_max]] = A[:, [j_max, k]]
            col_order[[k, j_max]] = col_order[[j_max, k]]
            det *= -1

        for i in range(k+1, n):
            m = A[i, k] / A[k, k]
            A[i, k:] -= m * A[k, k:]
            b[i] -= m * b[k]

        det *= A[k, k]

    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = (b[i] - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]

    x_final = np.zeros_like(x)
    for i in range(n):
        x_final[col_order[i]] = x[i]

    return x_final, det


In [3]:
def evaluate_functions(func_strings, x):
    values = []
    scope = {f"x{i}": x[i] for i in range(len(x))}
    scope["np"] = np

    for eq in func_strings:
        values.append(eval(eq, {"__builtins__": {}}, scope))

    return np.array(values, dtype=float)

In [4]:
# (f(x+h) - f(x-h)) / 2h
def jacobi(func_strings, x, h=1e-4):
    n = len(x)
    m = len(func_strings)
    J = np.zeros((m, n))

    for i in range(n):
        x_forward_h = x.copy()
        x_forward_h[i] += h
        x_backward_h = x.copy()
        x_backward_h[i] -= h

        f_forward_h = evaluate_functions(func_strings, x_forward_h)
        f_backward_h = evaluate_functions(func_strings, x_backward_h)

        D_h = (f_forward_h - f_backward_h) / (2 * h)

        h2 = h / 2
        x_forward_h2 = x.copy()
        x_forward_h2[i] += h2
        x_backward_h2 = x.copy()
        x_backward_h2[i] -= h2

        f_forward_h2 = evaluate_functions(func_strings, x_forward_h2)
        f_backward_h2 = evaluate_functions(func_strings, x_backward_h2)

        D_h2 = (f_forward_h2 - f_backward_h2) / (2 * h2)

        J[:, i] = D_h2 + (D_h2 - D_h) / 3

    return J

In [9]:
def newton_system(func_strings, x0, eps=1e-6):
    x_curr = np.array(x0, dtype=float)
    n = len(x0)
    variables = [f"x{i}" for i in range(n)]
    
    k = 1
    while True:
        F = evaluate_functions(func_strings, x_curr)
        J = jacobi(func_strings, x_curr)
        try:
            delta, det = func(J, -F)
        except ValueError:
            print("Jacobian is singular! Newton's method cannot continue.")
            return None
        
        x_curr += delta
        
        if np.linalg.norm(delta, ord=np.inf) < eps:
            print("\nРезультат знайдено!")
            print(f"x    = {np.array2string(x_curr)}")
            print(f"Кількість ітерацій: {k}")

            try:
                with open("Newton_result.txt", "w", encoding="utf-8") as f:
                    f.write(f"Кількість ітерацій: {k}\n")
                    
                    f.write("Знайдений вектор коренів (X):\n")
                    for i, val in enumerate(x_curr):
                        f.write(f"  x{i} = {val:.10f}\n")
                    
                    f.write("\nПеревірка (підстановка у систему f(x)):\n")
                    residuals = evaluate_functions(func_strings, x_curr)
                    f.write(f"  {np.array2string(residuals, precision=4, floatmode='fixed')}\n")
                
            except IOError as e:
                print(f"Помилка при записі файлу: {e}")
            return x_curr
        
        k += 1

In [10]:
with open("Newton_input.txt", "r") as f:
    lines = f.read().splitlines()

x0 = list(map(float, lines[0].split()))
equations = lines[1:]

root = newton_system(equations, x0)


Результат знайдено!
x    = [1.2 5.6 4.3 1. ]
Кількість ітерацій: 9


In [7]:
'''
1 1 1 1
x0 + 2*x1 + x2 + 4*x3 - 20.700
x0**2 + 2*x0*x1 + x3**3 - 15.880
x0**3 + x2**2 + x3 - 21.218
3*x1 + x2*x3 - 21.100
'''

'''
1 1 1
x0 + 2*x1 + x2 - 20.700
x0**2 + 2*x0*x1 - 15.880
x0**3 + x2**2 - 21.218
'''

'''
1 1
x0 + 2*x1 - 20.700
x0**2 + 2*x0*x1 - 15.880
'''

''

''